In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install pyforest
import pyforest
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")
df

In [ ]:
dfethnicity = df.groupby('ethnicity').count()

In [ ]:
ditribution_plot(x=ethnicity_dist.index, y=ethnicity_dist.values, name='Ethnicity Distribution')

In [ ]:
plt.hist(df['ethnicity'])

In [ ]:
for i in range(0,23705):
  
  df['pixels'][i] = np.array(df['pixels'][i].split(' '), dtype=np.float)
       
df['pixels']

In [ ]:
df['pixels'] = df['pixels']/255.0
df

In [ ]:
pd.set_option('display.max_rows', 120)
df.groupby('age').count()

In [ ]:
df['age'] = np.where(df['age'] > 80,81, df['age'])
df.groupby('age').count()

In [ ]:
df = df.sample(frac = 1) 
df

In [ ]:
cols = ['age','ethnicity','gender']
y = df[cols]
y

In [ ]:
X = np.stack(np.array(df['pixels']),axis=0) 
X.shape

In [ ]:
X = X.reshape(23705,48,48,1)

In [ ]:
X.shape

In [ ]:
plt.matshow(X[2])

In [ ]:
df.drop(columns = 'img_name', inplace=True)
df

In [ ]:
y_gender = np.array(y['gender'])
y_ethnicity = np.array(y['ethnicity'])
y_age = np.array(y['age'])

In [ ]:

import tensorflow as tf
import keras


In [ ]:
X_gender_train, X_gender_test, y_gender_train, y_gender_test = train_test_split(X, y_gender, train_size=0.7, random_state=42)
X_ethnicity_train, X_ethnicity_test, y_ethnicity_train, y_ethnicity_test = train_test_split(X, y_ethnicity, train_size=0.7, random_state=42)
X_age_train, X_age_test, y_age_train, y_age_test = train_test_split(X, y_age, train_size=0.7, random_state=42)

In [ ]:
def build_model(num_classes, activation, loss, metrics):
    
    inputs = tf.keras.Input(shape=(48, 48, 1))
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D(2)(x)
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)
    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D(2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation=activation)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='adam',
        loss=loss,
        metrics=[metrics]
    )
    
    return model

In [ ]:
gender_model = build_model(1, activation='sigmoid', loss='binary_crossentropy', metrics='accuracy')

gender_history = gender_model.fit(
    X_gender_train,
    y_gender_train,
    validation_split=0.2,
    batch_size=64,
    epochs=7,
    verbose=1
)

In [ ]:
plt.plot(gender_history.history['loss'])
plt.plot(gender_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
gender_acc = gender_model.evaluate(X_gender_test, y_gender_test)

# serialize model to JSON
gender_model_json = gender_model.to_json()
with open("gender_model.json", "w") as json_file:
    json_file.write(gender_model_json)
# serialize weights to HDF5
gender_model.save_weights("gender_model.h5")
print("Saved model to disk")

In [ ]:
 age_model = build_model(1, activation='linear', loss='mae', metrics='mse')

 age_history = age_model.fit(
     X_age_train,
     y_age_train,
     validation_split=0.2,
     batch_size=64,
     epochs=25,
     verbose=1
 )

In [ ]:
plt.plot(age_history.history['loss'])
plt.plot(age_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
age_acc = age_model.evaluate(X_age_test, y_age_test)

In [ ]:
y_age_test[:20]

In [ ]:
y_pred = age_model.predict(X_age_test[:20])
y_pred

In [ ]:
x = X_age_test[3]
x.shape

In [ ]:
x=np.expand_dims(x,axis=0)
x.shape

age_model_json = age_model.to_json()
with open("age_model.json", "w") as json_file:
    json_file.write(age_model_json)
# serialize weights to HDF5
age_model.save_weights("age_model.h5")
print("Saved model to disk")

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
age_model.predict(x)

In [ ]:
 ethnicity_model = build_model(5, activation='softmax', loss='sparse_categorical_crossentropy', metrics='accuracy')

 ethnicity_history = ethnicity_model.fit(
     X_ethnicity_train,
     y_ethnicity_train,
     validation_split=0.2,
     batch_size=64,
     epochs=7,
     verbose=1
 )

In [ ]:
plt.plot(ethnicity_history.history['loss'])
plt.plot(ethnicity_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
ethnicity_acc = ethnicity_model.evaluate(X_ethnicity_test, y_ethnicity_test)

# serialize model to JSON
ethnicity_model_json = ethnicity_model.to_json()
with open("ethnicity_model.json", "w") as json_file:
    json_file.write(ethnicity_model_json)
# serialize weights to HDF5
ethnicity_model.save_weights("ethnicity_model.h5")
print("Saved model to disk")